In [ ]:
import os
import pandas as pd
import config
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
def extract_features_labels(data):
    y = data['label'].values
    X = data.drop(columns=['label']).values
    return X, y
def get_data(file_name):
    file_path = os.path.join(config.DATA_DIR, file_name)


    if 'train' in file_name:
        data_train = pd.read_csv(file_path)  # 修正为使用 file_path
        X_train, y_train = extract_features_labels(data_train)
        test_file_path = file_path.replace("train", "test")
        data_test = pd.read_csv(test_file_path)
        X_test, y_test = extract_features_labels(data_test)
    else:
        data = pd.read_csv(file_path)
        label = data['label']
        data = data.drop(columns=['label'])
        X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.3, random_state=22)
        # X_train, y_train, X_test, y_test = X_train.values, y_train.values, X_test.values, y_test.values

    # sc = StandardScaler()
    # X_train = sc.fit_transform(X_train)
    # X_test = sc.transform(X_test)
    X_train = pd.DataFrame(X_train)
    X_test = pd.DataFrame(X_test)
    num_samples = X_train.shape[0] + X_test.shape[0]
    num_features = X_train.shape[1]  
    size = (num_samples, num_features)
    print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)
    return X_train, y_train, X_test, y_test, size

def get_X_Y(file_path):
    X_train, y_train, X_test, y_test, size = get_data(file_path)
    if isinstance(X_train, pd.DataFrame) and isinstance(X_test, pd.DataFrame):
        X_train = pd.DataFrame(X_train)
        X_test = pd.DataFrame(X_test)
    if isinstance(y_train, pd.Series) and isinstance(y_test, pd.Series):
        y_train = pd.Series(y_train)
        y_test = pd.Series(y_test)  
    X = pd.concat([X_train, X_test], axis=0)
    y = pd.concat([y_train, y_test], axis=0)
    return X, y, size

In [ ]:
import pandas as pd
import os
import numpy as np
import fcntl
import json

In [ ]:
def save_json(file, accs, type):
    file_path = "/home/xiaowenyuan/Python/MGFS/picture/result/result.json"
    if os.path.exists(file_path):
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            if type in data:
                data[type][file] = accs
            else:
                data[type] = {file: accs}
            with open(file_path, 'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=4)
        except Exception as e:
            print(f"读取或写入文件 {file_path} 时出错: {e}")
    else:
        try:
            with open(file_path, 'w', encoding='utf-8') as f:
                data = {type: {file: accs}}
                json.dump(data, f, ensure_ascii=False, indent=4)
        except Exception as e:
            print(f"创建并写入文件 {file_path} 时出错: {e}")
def save_result_csv(file_name, algorithm, classifier, size, dataSet, acc, f1, recall, selected_features,time=None):
    # 打开文件并获取文件描述符
    with open(file_name, 'a+') as file:
        try:
            # 尝试获取文件的排他锁，确保其他程序无法同时写入
            fcntl.flock(file.fileno(), fcntl.LOCK_EX)
            file.seek(0)  # 将文件指针移到文件开头
            if file.read(1):  # 检查文件是否有内容
                file.seek(0)  # 再次将文件指针移到文件开头
                data = pd.read_csv(file)
                new_result = {
                    'algorithm': algorithm,
                    'Classiflier': classifier,
                    'rows': size[0],
                    'columns': size[1],
                    'dataSet': dataSet,
                    'Accuracy': acc,  # 将标量转为列表
                    'F1 Score': f1,
                    'Recall': recall,
                    'featureLength': len(selected_features),
                    'features': selected_features
                }
                if time is not None:
                    new_result['time']=time
                results = pd.concat(
                    [data, pd.DataFrame([new_result])],
                    ignore_index=True
                )
            else:
                results = pd.DataFrame({
                    'algorithm': [algorithm],
                    'Classiflier': [classifier],
                    'rows': size[0],
                    'columns': size[1],
                    'dataSet': [dataSet],
                    'Accuracy': [acc],  # 将标量转为列表
                    'F1 Score': [f1],
                    'Recall': [recall],
                    'featureLength': [len(selected_features)],
                    'features': [selected_features]
                })
                if time is not None:
                    results['time']=[time]
            # 保存结果到文件
            results.to_csv(file_name, index=False)
        finally:
            # 释放文件锁
            fcntl.flock(file.fileno(), fcntl.LOCK_UN)

In [ ]:
import numpy as np
import pandas as pd
import random


from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import SpectralClustering
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import train_test_split
from minepy import MINE

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch_geometric.nn import GATConv, GCNConv

from torch.utils.data import DataLoader
import timeit

from net import Autoencoder, GAT, DQNAgent, Generator, Discriminator
from util import PriorityReplayBuffer
import warnings
from sklearn.linear_model import LogisticRegression
warnings.filterwarnings("ignore")
torch.backends.cudnn.benchmark = True
from sklearn.ensemble import ExtraTreesClassifier
# clf = ExtraTreesClassifier(n_estimators=50, n_jobs=-1) #ExtraTreesClassifier(n_estimators=50, n_jobs=-1)极端随机树
from sklearn.ensemble import RandomForestClassifier
# clf = RandomForestClassifier(n_estimators=100, max_depth=10, n_jobs=-1)
from sklearn.svm import SVC
# clf = SVC(kernel='rbf', probability=True)
from xgboost import XGBClassifier
# clf = XGBClassifier(n_estimators=100, max_depth=3, learning_rate=0.1, n_jobs=-1)

import os
from sklearn.metrics import f1_score, recall_score
from save_result import save_result_csv
from read_csv import get_data
import config

device = config.DEVICE
torch.autograd.set_detect_anomaly(True)

In [ ]:
def compute_mi(X, n_features):
    """MI矩阵计算（特征数×特征数）"""
    print("开始计算MI>>>>>>>")
    mi_matrix = np.zeros((n_features, n_features))
    discrete_flags = [np.unique(X[:, i]).size < 10 for i in range(n_features)]  # 提前计算离散标志
    # print(f"离散特征标志计算完成:{discrete_flags}")
    for i in range(n_features):
        # print(f"正在计算第{i}个特征的MI")
        mi = mutual_info_regression(X, X[:, i], discrete_features=discrete_flags, n_jobs=-1)
        mi_matrix[i, :] = mi

    mi_matrix = (mi_matrix + mi_matrix.T) / 2  # 对称化
    print("计算结束<<<<<<<<")
    return np.nan_to_num(mi_matrix)


def feature_clustering_mi(mi_matrix, n_groups=15):
    """鲁棒的谱聚类"""
    mi_matrix = np.maximum(mi_matrix, 0)
    mi_matrix = (mi_matrix + mi_matrix.T) / 2
    
    try:
        clustering = SpectralClustering(
            n_clusters=n_groups,
            affinity='precomputed',
            random_state=42
        ).fit(mi_matrix)
    except Exception as e:
        print(f"谱聚类出错: {e}")
        n_features = mi_matrix.shape[0]
        return [list(range(i, min(i + 22, n_features))) for i in range(0, n_features, 22)]
    
    feature_groups = [[] for _ in range(n_groups)]
    for idx, label in enumerate(clustering.labels_):
        feature_groups[label].append(idx)
    return feature_groups


def compute_mic(X, n_features):
    """MIC矩阵计算（特征数×特征数）"""
    mic_matrix = np.zeros((n_features, n_features))
    mine = MINE(alpha=0.6, c=15)

    if isinstance(X, pd.DataFrame):
        X = X.values
    
    for i in range(n_features):
        for j in range(i, n_features):
            if np.std(X[:, i]) == 0 or np.std(X[:, j]) == 0:
                mic = 0.0
            else:
                try:
                    mine.compute_score(X[:, i], X[:, j])
                    mic = mine.mic()
                except Exception as e:
                    print(f"计算MIC出错: {e}")
                    mic = 0.0
            mic_matrix[i, j] = mic_matrix[j, i] = mic

    return np.nan_to_num(mic_matrix)


def feature_clustering(mic_matrix, n_groups):
    """鲁棒的谱聚类"""
    mic_matrix = np.maximum(mic_matrix, 0)
    mic_matrix = (mic_matrix + mic_matrix.T) / 2
    
    try:
        clustering = SpectralClustering(
            n_clusters=n_groups,
            affinity='precomputed',
            random_state=42
        ).fit(mic_matrix)
    except Exception as e:
        print(f"谱聚类出错: {e}")
        n_features = mic_matrix.shape[0]
        return [list(range(i, min(i + 22, n_features))) for i in range(0, n_features, 22)]
    
    feature_groups = [[] for _ in range(n_groups)]
    for idx, label in enumerate(clustering.labels_):
        feature_groups[label].append(idx)
    return feature_groups

In [ ]:
class SumTree:
    def __init__(self, capacity):
        self.capacity = capacity
        self.tree = np.zeros(2 * capacity - 1)
        self.data = np.zeros(capacity, dtype=object)
        self.write_idx = 0

    def _propagate(self, idx, change):
        parent = (idx - 1) // 2
        self.tree[parent] += change
        if parent != 0:
            self._propagate(parent, change)

    def add(self, priority, data):
        idx = self.write_idx + self.capacity - 1
        self.data[self.write_idx] = data 
        self.update(idx, priority)
        self.write_idx = (self.write_idx + 1) % self.capacity

    def update(self, idx, priority):
        change = priority - self.tree[idx]
        self.tree[idx] = priority
        self._propagate(idx, change)

    def get_leaf(self, v):
        idx = 0
        while True:
            left = 2 * idx + 1
            if left >= len(self.tree):
                break
            if v <= self.tree[left]:
                idx = left
            else:
                v -= self.tree[left]
                idx = left + 1
        return self.data[idx - self.capacity + 1]


class PriorityReplayBuffer:
    def __init__(self, real_capacity, gen_capacity, alpha=1, beta=0.5):
        self.real_tree = SumTree(real_capacity)
        self.gen_tree = SumTree(gen_capacity)
        self.alpha = alpha
        self.beta = beta
        
    def add_real(self, transition, priority):
        self.real_tree.add(priority ** self.alpha, transition)

    def add_gen(self, transition, priority):
        self.gen_tree.add(priority ** self.alpha, transition)

    def get_idx(self):
        return self.real_tree.write_idx

    def sample(self, batch_size, p=0.5):
        real_batch_size = int(batch_size * p)
        gen_batch_size = batch_size - real_batch_size

        # 真实经验采样
        real_total = self.real_tree.tree[0]
        gen_total = self.gen_tree.tree[0]
        total = real_total + gen_total + 1e-5

        real_segment = real_total / total
        real_batch = [self.real_tree.get_leaf(np.random.uniform(0, real_segment)) for _ in range(real_batch_size)]

        # 生成经验采样
        gen_batch = [self.gen_tree.get_leaf(np.random.uniform(real_segment, 1)) for _ in range(gen_batch_size)]

        return real_batch + gen_batch

    def sample_true(self, batch_size):
        real_total = self.real_tree.tree[0]
        return [self.real_tree.get_leaf(np.random.uniform(0, real_total)) for _ in range(batch_size)]

In [ ]:
class FeatureSelectionEnv:
    def __init__(self,  X, y, X_test, y_test):  
        # 数据存储
        self.X = torch.FloatTensor(X.T).to(device)  # [特征数, 样本数]，移动到GPU
        self.y = y
        self.X_test,self.y_test = X_test,y_test
        self.cache = {}  # 用于存储特征选择结果的缓存
        self.num_features, self.num_samples = self.X.shape
        # self.feature_groups = feature_groups
        
        # self.mi_matrix = mi_matrix  # 存储MI矩阵
        # self.pre_edge_index = self._build_graph(mi_matrix)
        
        # 初始化模型，并移动到GPU
        self.ae = Autoencoder(self.num_features, self.num_samples).to(device)
        # self.gat = GAT(64).to(device)
        self.gat = GAT(64).to(device)
        
        # 预训练与验证
        self._pretrain_ae()
        self.z_ae, self.d_ae = self.ae(self.X)
        print(f"z_ae shape: {self.z_ae.shape}")  # 打印z_ae的形状
        self.z_ae = self.z_ae.detach()  # 分离梯度，避免更新
        self.X_ae = self.z_ae.T.cpu()
        self.mi_matrix = compute_mi(self.X_ae, X.shape[1])
        laplacian = np.diag(np.sum(self.mi_matrix, axis=1)) - self.mi_matrix
        eigvals = np.sort(np.linalg.eigvalsh(laplacian))
        
        # 寻找特征值拐点（经验法则）
        eig_diff = np.diff(eigvals[:20])
        n_groups = np.argmax(eig_diff) + 1  # 加1因为diff后索引偏移
        # n_groups_num = X_train.shape[1] // 5 + 1
        if n_groups < X_train.shape[1] // 10 + 1:
            n_groups = X_train.shape[1] // 5 + 1
        print(f"特征分组数量: {n_groups}")
        self.feature_groups = feature_clustering_mi(self.mi_matrix, n_groups)
        self.n_agents = len(self.feature_groups)
        # print(f"z_ae shape: {self.z_ae.shape}")  # 打印z_ae的形状

    def _build_graph(self, feature_select, top_k=5):
        """基于MI矩阵和已选特征的图结构构建"""
        edge_indices = []
        
        if len(feature_select) == 0:
            # 原有互信息拓扑连接
            for i in range(self.num_features):
                sorted_indices = np.argsort(self.mi_matrix[i])[::-1]
                valid_neighbors = [
                    j for j in sorted_indices 
                    if j != i and j < self.mi_matrix.shape[0]
                ][:top_k]
                edge_indices.extend([[i, j] for j in valid_neighbors])
        else:
            # 仅对已选特征进行构图
            for i in feature_select:
                sorted_indices = np.argsort(self.mi_matrix[i])[::-1]
                # 筛选出同样在已选特征中的邻居
                valid_neighbors = [
                    j for j in sorted_indices 
                    if j != i and j in feature_select
                ][:top_k]
                edge_indices.extend([[i, j] for j in valid_neighbors])
        
        # print(f"最终边数量: {len(edge_indices)} (含组内全连接)")  # 修正打印边数量的错误
        edge_tensor = torch.LongTensor(edge_indices).t().contiguous().to(device)
        return edge_tensor
    def _pretrain_ae(self, epochs=50):
        """带梯度裁剪的预训练"""
        optimizer = optim.AdamW(self.ae.parameters(), lr=1e-3)
        criterion = nn.MSELoss()  # 提前定义损失函数
        for _ in range(epochs): 
            _, recon = self.ae(self.X)
            loss = criterion(recon, self.X)
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.ae.parameters(), 5.0)
            optimizer.step()

    def get_state(self, feature_select):
        with torch.no_grad():
            edge_index = self._build_graph(feature_select)
            gat_out = self.gat(self.z_ae, edge_index)  # [num_features, hidden_dim]
            # print(gat_out.shape)
        return gat_out.mean(dim=0)  # [hidden_dim]

    def calculate_reward(self, selected_features, data_loader, clf):
        if len(selected_features) == 0:
            return -1.0, 0.0
        selected_features = sorted(selected_features)
        if str(selected_features) in self.cache:
            reward, acc = self.cache[str(selected_features)]
            return reward, acc
        else:
            acc = 0.0
            try:
                # 使用 enumerate 的第二个参数 start=0 明确起始索引，遍历不超过 MAX_BATCHES 个批次的数据
                for i, (batch_X, batch_y) in enumerate(data_loader, start=0):
                    batch_X_selected = batch_X[:, selected_features]
                    
                    # 训练分类器并计算准确度
                    clf.fit(batch_X_selected, batch_y)
                    # y_pred = clf.predict(self.X_test[:, selected_features])
                    # acc = accuracy_score(self.y_test, y_pred)
                    acc = clf.score(batch_X_selected, batch_y)
            except Exception as e:  # 明确捕获异常类型
                print(f"z_ae 形状: {self.z_ae.shape}")
                print(f"LogisticRegression 训练出错: {e}")
            redundancy = sum(
                np.mean(self.mi_matrix[group][:, group]) 
                for group in self.feature_groups 
                if len(group) > 1
            ) / self.num_features
            # reward = 0.8 * acc + 0.2 * (0.5 * (1 - redundancy) + 0.5 * (1 - len(selected_features) / self.num_features))
            redundancy_penalty = 1 - redundancy
            length_penalty = 1 - len(selected_features) / self.num_features
            reward = 0.9 * acc + 0.1 * redundancy_penalty 
            self.cache[str(selected_features)] = (reward, acc)
            return reward, acc

In [ ]:
class MultiAgentSystem:
    def __init__(self, env):
        self.env = env
        # 初始化智能体并移动到GPU
        self.agents = [(DQNAgent(env.gat.output_dim).to(device), DQNAgent(env.gat.output_dim, False).to(device)) for _ in range(env.n_agents)]
        self.update_target = 10
        self.optimizers = [optim.Adam(agent[0].parameters(), lr=1e-4) for agent in self.agents]
        # 初始化生成器和判别器并移动到GPU
        self.generators = [Generator(env.gat.output_dim, 1).to(device) for _ in range(env.n_agents)]
        self.discriminators = [Discriminator(env.gat.output_dim, 1).to(device) for _ in range(env.n_agents)]
        self.gamma = 0.99
        self.batch_size = 32
        # self.buffer_tree = PriorityReplayBuffer(100)
        self.train_GAN_batch_size = 64
        self.TARGET_REPLACE_ITER = 50
        self.loss_func = nn.MSELoss()
        self.data_loader = DataLoader(torch.utils.data.TensorDataset(
                self.env.X.T.cpu(),  # 转换为 [样本数, 特征数] 并移动到 CPU
                torch.tensor(self.env.y, dtype=torch.float32).cpu()  # 确保标签是张量并移动到 CPU
            ), batch_size=256, shuffle=True)
    def generate_and_add_fake_data(self, z_size):
        for agent_idx in range(self.env.n_agents):
            z = torch.randn(z_size, 32).to(device)  # 移动到GPU
            fake_datas = self.generators[agent_idx](z)
            split_sizes = [self.env.gat.output_dim, 1, 1, self.env.gat.output_dim]
            s_gen_all, a_gen_all, r_gen_all, s_next_gen_all = fake_datas.split(split_sizes, dim=1)
            a_gen_all = (a_gen_all > 0.5).int()
            # with torch.no_grad():
            #     # 计算生成数据的TD误差
            #     q_values = self.agents[agent_idx][0](s_gen_all)
            #     next_q_values = self.agents[agent_idx][1](s_next_gen_all)
            #     td_errors = torch.abs(r_gen_all + self.gamma * next_q_values.max(dim=1)[0].unsqueeze(1) - q_values.gather(1, a_gen_all.long()))
                
                # 对每个样本计算优先级
            priorities = (r_gen_all + 1e-5)
            transitions = [self._to_transition(s_gen_all[i], r_gen_all[i], int(a_gen_all[i]), s_next_gen_all[i]) for i in range(z_size)]
            for i in range(z_size):
                transition = transitions[i]
                self.agents[agent_idx][0].memory.add_gen(transition, priorities[i])
    def train_with_gan(self, clf, dqn_episodes=500):
        epsilon = 1.0
        epsilon_decay = 0.98
        min_epsilon = 0.05
        state = self.env.get_state([]).to(device)  # 移动到GPU
        Flag = True

        group_features = {
            i: set(group) 
            for i, group in enumerate(self.env.feature_groups)
        }
        results = {"reward":0, "episode":0, "features":[], "acc":0}
        for episode in range(dqn_episodes):
            actions = []
            while 1 not in actions:
                actions = [eval_agent.act(state, epsilon) for (eval_agent, target_agent) in self.agents]
            selected_features = [f for group, action in zip(self.env.feature_groups, actions) if action == 1 for f in group]

            full_features = set(selected_features)
          
            # 随机选取最多 10 个批次并合并
            data_loader_list = list(self.data_loader)
            selected_batches = random.sample(data_loader_list, min(10, len(data_loader_list)))
            if selected_batches:
                merged_X, merged_y = zip(*selected_batches)
                merged_X = torch.cat(merged_X, dim=0)
                merged_y = torch.cat(merged_y, dim=0).to(torch.int32)
                selected_batches = [(merged_X, merged_y)]

            # 直接使用选取的批次调用 calculate_reward
            full_reward, acc = self.env.calculate_reward(list(full_features), selected_batches, clf)
            next_state = self.env.get_state(selected_features).to(device)  # 移动到GPU
            if acc > results["acc"] or full_reward > results["reward"]:
                results["reward"] = full_reward
                results["episode"] = episode
                results["acc"] = acc
                results["features"] = selected_features
            for agent_idx in range(self.env.n_agents):
                # 移除当前智能体所选特征的集合
                without_agent_features = full_features - group_features[agent_idx]
                # 计算包含和不包含当前智能体所选特征时的奖励
                without_reward, _ = self.env.calculate_reward(list(without_agent_features),selected_batches, clf)
                # 计算边缘奖励
                marginal_reward = full_reward - without_reward
                # # marginal_rewards.append(marginal_reward)
                # with torch.no_grad():
                #     q_values = self.agents[agent_idx][0](state)
                #     next_q_values = self.agents[agent_idx][1](next_state)
                #     td_error = torch.abs(marginal_reward + self.gamma * next_q_values.max() - q_values.max())
                priorities = (marginal_reward + 1e-5)
                transition = self._to_transition(state, marginal_reward, actions[agent_idx], next_state)  # 使用边缘特征奖励
                self.agents[agent_idx][0].memory.add_real(transition, priorities)

            state = next_state

            if episode > 100 :
                #一百次正常训练后，开始训练GAN
                if Flag:
                    
                    self.train_GAN()
                    Flag = False
                    # 填满整棵树
                    self.generate_and_add_fake_data(self.batch_size * 7)
                else:
                    self.generate_and_add_fake_data(self.batch_size)
            if episode > self.batch_size:
                self._update_models_tree(Flag, episode)
            epsilon = max(min_epsilon, epsilon * epsilon_decay)
            print(f"Episode {episode:03d} | Reward: {full_reward:.8f} | Features: {len(selected_features)} | ACC: {acc:.8f}")
        return results

    # 使用经验管理进行更新
    def _update_models_tree(self, flag, episode):

        should_update_target = episode % self.TARGET_REPLACE_ITER == 0

        # 批量处理所有智能体的更新
        for agent_idx, (online_agent, target_agent) in enumerate(self.agents):
            if flag:
                transitions = online_agent.memory.sample_true(self.batch_size)
            else:
                transitions = online_agent.memory.sample(self.batch_size)

            # try:
            # 提前将数据转换为列表，避免多次遍历 transitions
            states = [x[0] for x in transitions]
            actions = [x[1] for x in transitions]
            rewards = [x[3] for x in transitions]
            next_states = [x[2] for x in transitions]

            # 一次性将数据转换为张量并移动到指定设备
            s = torch.stack([torch.as_tensor(state, dtype=torch.float32) for state in states]).to(device)
            a = torch.stack([torch.as_tensor(action, dtype=torch.int64) for action in actions]).to(device)
            r = torch.stack([torch.as_tensor(reward, dtype=torch.float32) for reward in rewards]).to(device)
            s_next = torch.stack([torch.as_tensor(next_state, dtype=torch.float32) for next_state in next_states]).to(device)
        

            # 计算当前Q值
            q_values = online_agent(s)
            if a.dim() == 1:
                a = a.unsqueeze(-1)
            q_eval = q_values.gather(1, a)

            # 计算目标Q值，使用 no_grad 避免不必要的梯度计算
            with torch.no_grad():
                next_q_values = target_agent(s_next)
                max_next_q = next_q_values.max(dim=1, keepdim=True)[0]
                q_target = r.unsqueeze(-1) + self.gamma * max_next_q

            # 计算损失
            loss = self.loss_func(q_eval, q_target)

            # 反向传播和参数更新
            self.optimizers[agent_idx].zero_grad()  # 先清零梯度
            loss.backward(retain_graph=True)
            # 梯度裁剪，防止梯度爆炸
            torch.nn.utils.clip_grad_norm_(online_agent.parameters(), max_norm=1.0)
            self.optimizers[agent_idx].step()

            # 更新目标网络
            if should_update_target:
                # 使用参数过滤加载
                target_agent.load_state_dict(
                    {k: v for k, v in online_agent.state_dict().items() 
                     if 'generator' not in k and 'discriminator' not in k},
                    strict=False
                )

    # 转换保存格式        
    def _to_transition(self, state, reward, action, next_state):
        if isinstance(action, torch.Tensor):
            action = action[0].item()
        if isinstance(reward, torch.Tensor):
            reward = reward[0].item()
        return (state, action, next_state, reward)

    # 训练GAN网络
    def train_GAN(self):
        for agent_idx, (eval_agent, target_agent) in enumerate(self.agents):
            for _ in range(100):
                opt_gen = optim.Adam(self.generators[agent_idx].parameters(), lr=0.0001, betas=(0.5, 0.9))
                opt_dis = optim.Adam(self.discriminators[agent_idx].parameters(), lr=0.0001, betas=(0.5, 0.9))
                z = torch.randn(self.batch_size, 32).to(device)
                
                transitions = eval_agent.memory.sample_true(self.batch_size)
                # 添加数据校验
                valid_transitions = [
                    x for x in transitions 
                    if isinstance(x, (tuple, list)) and len(x) >=4
                ]
                if len(valid_transitions) == 0:
                    print("警告：缓冲区无有效数据，跳过GAN训练")
                    return
                    
                try:
                    # 使用更安全的转换方式
                    s_rea = torch.stack([torch.as_tensor(x[0], dtype=torch.float32).to(device) for x in valid_transitions])
                    a_rea = torch.stack([torch.as_tensor(x[1], dtype=torch.int).to(device) for x in valid_transitions])
                    r_rea = torch.stack([torch.as_tensor(x[3], dtype=torch.float32).to(device) for x in valid_transitions])
                    s_next_rea = torch.stack([torch.as_tensor(x[2], dtype=torch.float32).to(device) for x in valid_transitions])
                except Exception as e:
                    print(f"数据转换异常: {e}")
                    print("问题数据样本:", [x[:2] for x in transitions[:3]])  # 打印前3个样本的前两个元素
                    return

                split_sizes = [
                    self.env.gat.output_dim,
                    1, 
                    1, 
                    self.env.gat.output_dim
                ]

                # 生成假数据
                fake_data = self.generators[agent_idx](z).detach() 
                s_gen, a_gen, r_gen, s_next_gen = fake_data.split(split_sizes, dim=1)
                
                self.generators[agent_idx].eval()  # 固定生成器
                self.discriminators[agent_idx].train()  # 启用判别器梯度
                
                real_validity = self.discriminators[agent_idx](s_rea, a_rea, r_rea, s_next_rea)
                fake_validity = self.discriminators[agent_idx](s_gen.detach(), a_gen.detach(), r_gen.detach(), s_next_gen.detach())
                
                loss_dis = -torch.mean(real_validity) + torch.mean(fake_validity)
                
                opt_dis.zero_grad()
                loss_dis.backward(retain_graph=True)  # 保留计算图
                torch.nn.utils.clip_grad_norm_(self.discriminators[agent_idx].parameters(), 1.0)  # 梯度裁剪
                opt_dis.step()
                
                self.generators[agent_idx].train()  # 启用生成器梯度
                self.discriminators[agent_idx].eval()  # 固定判别器
                
                # 重新生成数据（保持梯度）
                fake_data_new = self.generators[agent_idx](z)
                s_gen_new, a_gen_new, r_gen_new, s_next_gen_new = fake_data_new.split(split_sizes, dim=1)
                
                fake_validity_new = self.discriminators[agent_idx](s_gen_new, a_gen_new, r_gen_new, s_next_gen_new)
                loss_gen = -torch.mean(fake_validity_new)
                
                opt_gen.zero_grad()
                loss_gen.backward()
                torch.nn.utils.clip_grad_norm_(self.generators[agent_idx].parameters(), 1.0)  # 梯度裁剪
                opt_gen.step()

In [ ]:
class Autoencoder(nn.Module):
    """特征级自动编码器"""
    def __init__(self, num_features, num_samples):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(num_samples, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 64)
        )
        self.decoder = nn.Sequential(
            nn.Linear(64, 256),
            nn.ReLU(),
            nn.Linear(256, num_samples)
        )

    def forward(self, x):
        z = self.encoder(x)
        return z, self.decoder(z)


class GAT(nn.Module):
    """图注意力网络"""
    def __init__(self, input_dim, hidden_dim=128, heads=4):
        super().__init__()
        self.gat1 = GATConv(input_dim, hidden_dim, heads=heads)
        self.gat2 = GATConv(hidden_dim * heads, hidden_dim, heads=1)
        self.output_dim = hidden_dim

    def forward(self, x, edge_index):
        x = nn.functional.elu(self.gat1(x, edge_index))
        return self.gat2(x, edge_index)

class GCN(nn.Module):
    """图卷积网络"""
    def __init__(self, input_dim, hidden_dim=128):
        super().__init__()
        self.gcn1 = GCNConv(input_dim, hidden_dim)
        self.gcn2 = GCNConv(hidden_dim, hidden_dim)
        self.output_dim = hidden_dim

    def forward(self, x, edge_index):
        x = self.gcn1(x, edge_index)
        x = F.relu(x)
        return self.gcn2(x, edge_index)

class DQNAgent(nn.Module):
    """决策智能体"""
    def __init__(self, state_dim, target=True, output_dim=2):
        super().__init__()
        if target:
            self.target = False
            self.memory = PriorityReplayBuffer(real_capacity=100,gen_capacity=100)
        self.output_dim = output_dim
        self.fc1 = nn.Linear(state_dim, 64)
        self.fc1.weight.data.normal_(0, 0.1)

        self.relu = nn.ReLU()

        self.fc2 = nn.Linear(64, output_dim)

        self.net = nn.Sequential(
            self.fc1,
            self.relu,
            self.fc2
        )

    def forward(self, state):
        return self.net(state)

    def act(self, state, epsilon):
        if np.random.rand() < epsilon:
            return np.random.randint(self.output_dim)
        with torch.no_grad():
            return torch.argmax(self(state)).item()

    
# GAN模块
# 生成器
class Generator(nn.Module):
    def __init__(self, state_dim, action_dim, noise_dim=32):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(noise_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, state_dim * 2 + action_dim * 2)
        )

    def forward(self, z):
        return self.net(z)


# 判别器
class Discriminator(nn.Module):
    def __init__(self, state_dim, action_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(state_dim * 2 + action_dim * 2, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, s, a, r, s_next):
        a = a.unsqueeze(1) if a.dim() == 1 else a
        r = r.unsqueeze(1) if r.dim() == 1 else r
        x = torch.cat([s, a, r, s_next], dim=1)
        return self.net(x)

In [ ]:
nodes = 5
for file_name in config.DATASET_NAMES:

    X_train, y_train, X_test, y_test, size = get_data(file_name)
    if isinstance(X_train, pd.DataFrame):
        X_train = X_train.values
    if isinstance(X_test, pd.DataFrame):
        X_test = X_test.values
    if isinstance(y_train, pd.Series):
        y_train = y_train.values
    if isinstance(y_test, pd.Series):
        y_test = y_test.values
    # y_train = y_train.values
    # X_test = X_test.values
    # y_test = y_test.values

    if X_train.shape[1]>4000:
        continue
    
    if X_train.shape[0] >3000000:
        device = torch.device("cpu")
    else:
        device = torch.device("cuda:1")
    print("==================数据读取结束==================")
    # print("====================数据处理====================")
    # mi_matrix = compute_mi(X_train, X_train.shape[1])
    # print(f"MI矩阵维度: {mi_matrix.shape} (应等于特征数平方)")
    # assert mi_matrix.shape == (X_train.shape[1], X_train.shape[1]), "MI矩阵维度错误"
    # laplacian = np.diag(np.sum(mi_matrix, axis=1)) - mi_matrix
    # eigvals = np.sort(np.linalg.eigvalsh(laplacian))
    
    # # 寻找特征值拐点（经验法则）
    # eig_diff = np.diff(eigvals[:20])
    # n_groups = np.argmax(eig_diff) + 1  # 加1因为diff后索引偏移
    # # n_groups_num = X_train.shape[1] // 5 + 1
    # if n_groups < X_train.shape[1] // 10 + 1:
    #     n_groups = X_train.shape[1] // 5 + 1
    # print(f"特征分组数量: {n_groups}")
    # feature_groups = feature_clustering_mi(mi_matrix, n_groups)
    # print(f"特征分组数量: {len(feature_groups)}")
    # print(feature_groups)
    # all_features = set([f for group in feature_groups for f in group])
    # assert len(all_features) == X_train.shape[1], "特征分组存在遗漏"
    # assert max(all_features) < X_train.shape[1], "存在非法特征索引"
    # print("==================数据处理结束==================")
    # 初始化环境
    env = FeatureSelectionEnv(X_train, y_train, X_test, y_test)
    
    print("====================开始训练====================")
    for clf_name, clf in config.CLASSFIERS.items():
        env.cache = {}
        print(f"====================开始训练：{clf_name}====================")
        # 训练系统
        mas = MultiAgentSystem(env)
        start = timeit.default_timer()
        result = mas.train_with_gan(clf, dqn_episodes=300)
        end = timeit.default_timer()
        print("====================处理结束====================")
        print("====================最终评估====================")

        # 最终评估
        selected_features = result["features"]
        clf.fit(X_train[:, selected_features], y_train)
        y_pred = clf.predict(X_test[:, selected_features])
        acc = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')
        
        print(f"\n最终结果: 选择{len(selected_features)}个特征, 测试准确率: {acc:.4f}, F1分数: {f1:.4f}, 召回率: {recall:.4f}")

        # 保存结果
        save_result_csv(config.RESULT_FILE,'MGFSBG',clf_name, size,file_name, acc, f1, recall, selected_features,time=end-start)